In [1]:
import pandas as pd
from openai import AzureOpenAI
import os
import random
from datetime import datetime, timedelta
import re

In [2]:
df = pd.read_parquet("synthetic_229_corrected.parquet")
df

,description,summary
0,sokorsak: Dyspné \nkon: K \nalder: 58 \nsocial...,"*Sjukdomshistoria*\nHypertoni, hypotyreos, hyp..."
1,sokorsak: Dyspné \nkon: K \nalder: 58 \nsocial...,<Sjukdomshistoria>\n58-årig kvinnlig patient m...
2,\nsökorsak: Ospecifik buksmärta \n kön: M \...,\n*Sjukdomshistoria*\n63-årig man med diabetes...
3,sokorsak: BrastSmarta \nkon: M \nalder: 51 \ns...,"\n*Sjukdomshistoria*\nHypertoni, Typ 2 diabete..."
4,sokorsak: BukSmarta \nkon: K \nalder: 20 \nsoc...,*Sjukdomshistoria*\nMisstänkt appendicit (K379...
...,...,...
224,\nsokorsak: Buksmärtor\nkon: K\nalder: 45\nsoc...,"**Sjukdomshistoria**\n• 45-årig kvinna, tidiga..."
225,\nsokorsak: Bröstsmärtor\nkon: K\nalder: 68\ns...,*Sjukdomshistoria*\n• Kvinnan är 68 år och har...
226,\nsokorsak: Återkommande huvudvärk\nkon: M\nal...,*Sjukdomshistoria*\n• Allergisk astma behandla...
227,\nsokorsak: Yrsel och huvudvärk\nkon: M\nalder...,*Sjukdomshistoria*\n• Ischemisk hjärtsjukdom m...


In [3]:
client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_ENDPOINT"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [4]:
instructions = """You are a helpful assistant, that helps to create synthetic data. 
                  You will be provided with a medical note of a patient and a summary.
                  The description and the summary are both in Swedish, and you should
                  answer in Swedish.
                  Please generate another note-summary pair for a new, realistic fake patient.
                  Remember to use the same template as the provided example for the summary.
                  """

In [5]:
diseases = []
with open("diseases.txt") as f:
    for line in f:
        diseases.append(line)
diseases = pd.DataFrame(diseases)
diseases

,0
0,Abdominal aortic aneurysm\n
1,Acne\n
2,Acute cholecystitis\n
3,Acute lymphoblastic leukaemia\n
4,Acute lymphoblastic leukaemia: Children\n
...,...
329,Warts and verrucas\n
330,Whooping cough\n
331,Wilms’ tumour\n
332,Womb (uterus) cancer\n


In [6]:
responses = []

In [ ]:
for i,(x,y) in df.iterrows():
    
    now = datetime.now()
    one_hour_later = now + timedelta(hours=2)
    current_time = one_hour_later.strftime("%H:%M:%S")
    
    dis = diseases.sample()[0].item().replace("\n","")
    #timeline = random.choice(["ongoing","2 years ago","5 years ago","10 years ago"])
    
    print(i,"-",current_time, "-", dis)
    
    question = f"""Please create a similar description-summary pair for a new, 
    fake patient with the following conditions:'{dis} : '
    <Description>{x}</Description><Summary>{y}</Summary>
    Respond with the same template, using tags."""
    
    messages = [{"role": "system", "content": instructions}, {"role": "user", "content": question}]
    
    response = client.chat.completions.create(
      model="gpt4",
      #response_format={ "type": "json_object" },
      messages=messages
    )
    
    match = re.split(r"<[^>]*>",response.choices[0].message.content)
    if len(match) == 5:
        desc = match[1]
        summ = match[3]
        responses.append((desc,summ))
    elif len(match) == 1:
        match = re.split(r"Sjukdomshistoria", txt,maxsplit=1)
        desc = match[0]
        summ = 'Sjukdomshistoria'+match[1]
        responses.append((desc,summ))
    
        
    

0 - 10:06:31 - Cerebral palsy
1 - 10:07:34 - Edwards’ syndrome
2 - 10:09:08 - Womb (uterus) cancer
3 - 10:10:06 - Ovarian cancer
4 - 10:11:13 - Gallbladder cancer
5 - 10:12:23 - Acute lymphoblastic leukaemia
6 - 10:13:30 - Acute lymphoblastic leukaemia
7 - 10:14:47 - Swollen glands
8 - 10:15:52 - Bronchitis
9 - 10:16:55 - Sjogren’s syndrome
10 - 10:18:26 - Vulval cancer
11 - 10:19:25 - Swollen glands
12 - 10:20:35 - Gastroenteritis
13 - 10:21:35 - Suicide
14 - 10:23:04 - Ovarian cancer: Teenagers and young adults
15 - 10:24:27 - Nosebleed
16 - 10:25:39 - Bladder cancer
17 - 10:26:57 - Edwards’ syndrome
18 - 10:28:04 - Non-Hodgkin lymphoma: Children
19 - 10:29:55 - Shingles
20 - 10:31:05 - Flu
21 - 10:32:36 - Nosebleed
22 - 10:33:33 - Bone cancer: Teenagers and young adults
23 - 10:34:34 - Addison’s disease
24 - 10:36:20 - Motor neurone disease (MND)
25 - 10:37:39 - Lipoedema
26 - 10:38:52 - Farting
27 - 10:40:19 - Oral thrush in adults
28 - 10:41:24 - Hand, foot and mouth disease
29 - 

In [12]:
responses = pd.DataFrame(responses,columns=["description","summary"])
responses

,description,summary
0,sokorsak: Balanssvårigheter och falltendens\nk...,"*Sjukdomshistoria*\nAstma, allergi mot björkpo..."
1,\nsökorsak: Menorragi och buksmärta \nkön: K \...,\n*Sjukdomshistoria*\n52-årig kvinna med hyper...
2,\nsokorsak: Oregelbundna blödningar \nkon: K \...,\n*Sjukdomshistoria*\nAstma. Familjehistorik m...
3,sokorsak: Ospecifik viktnedgång och trötthet \...,*Sjukdomshistoria*\nMisstanke om gallblåsecanc...
4,\nsokorsak: Trötthet och blåmärken \nkon: M \n...,"\n*Sjukdomshistoria*\n43-årig man, hälsosam ti..."
...,...,...
223,\nsokorsak: Yrsel och hörselnedsättning\nkon: ...,"**Sjukdomshistoria**\n• 52-årig man, hypertoni..."
224,\nsokorsak: Ihållande huvudvärk och konfusion\...,\n*Sjukdomshistoria*\n• Mannen är 72 år och ha...
225,\nsokorsak: Anhedoni och sömnstörningar\nkon: ...,\n*Sjukdomshistoria*\n• Ingen känd tidigare sj...
226,\nsokorsak: Andningssvårigheter och bröstsmärt...,\n*Sjukdomshistoria*\n• Hypertoni under behand...


In [14]:
joined = pd.concat([responses,df],ignore_index=True)
joined.to_parquet("synthetic_5th_april.parquet")

In [15]:
joined

,description,summary
0,sokorsak: Balanssvårigheter och falltendens\nk...,"*Sjukdomshistoria*\nAstma, allergi mot björkpo..."
1,\nsökorsak: Menorragi och buksmärta \nkön: K \...,\n*Sjukdomshistoria*\n52-årig kvinna med hyper...
2,\nsokorsak: Oregelbundna blödningar \nkon: K \...,\n*Sjukdomshistoria*\nAstma. Familjehistorik m...
3,sokorsak: Ospecifik viktnedgång och trötthet \...,*Sjukdomshistoria*\nMisstanke om gallblåsecanc...
4,\nsokorsak: Trötthet och blåmärken \nkon: M \n...,"\n*Sjukdomshistoria*\n43-årig man, hälsosam ti..."
...,...,...
452,\nsokorsak: Buksmärtor\nkon: K\nalder: 45\nsoc...,"**Sjukdomshistoria**\n• 45-årig kvinna, tidiga..."
453,\nsokorsak: Bröstsmärtor\nkon: K\nalder: 68\ns...,*Sjukdomshistoria*\n• Kvinnan är 68 år och har...
454,\nsokorsak: Återkommande huvudvärk\nkon: M\nal...,*Sjukdomshistoria*\n• Allergisk astma behandla...
455,\nsokorsak: Yrsel och huvudvärk\nkon: M\nalder...,*Sjukdomshistoria*\n• Ischemisk hjärtsjukdom m...
